# PHASE 1 PROJECT

- Name: Lilibeth Chepngetich Langat
- Student pace: full time
- Scheduled project review date/time: 3/10/2025
- Instructor name: Nikita Njoroge

In [1]:
# import needed libraries
import pandas as pd   # data wrangling
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
%matplotlib inline

In [3]:
# read csv file to pandas dataframe
main_aviation = pd.read_csv('./data/Aviation_Data.csv')
# check the first five rows
main_aviation.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [4]:
# check shape of the dataframe
main_aviation.shape

(90348, 31)

In [5]:
# check general information of the dataframe
main_aviation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [37]:
# check values inlocation dataset
main_aviation['Location'].value_counts()

ANCHORAGE, AK                            434
MIAMI, FL                                200
ALBUQUERQUE, NM                          196
HOUSTON, TX                              193
CHICAGO, IL                              184
                                        ... 
Devon, United Kingdom, United Kingdom      1
AMHERST, NH                                1
Dawson Springs, KY                         1
Loreto, Peru                               1
Ourgla, Antigua and Barbuda                1
Name: Location, Length: 27758, dtype: int64

In [40]:
# subset the dataframe to include only the columns we will need for the analysis
subset_aviation = main_aviation[['Country','Event.Date','Injury.Severity','Aircraft.damage','Aircraft.Category',
                                 'Make','Model','Number.of.Engines','Engine.Type','Purpose.of.flight',
                                 'Weather.Condition','Broad.phase.of.flight','Total.Fatal.Injuries',
                                 'Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured',]]
# check top rows in dataset
subset_aviation.head()

,Country,Event.Date,Injury.Severity,Aircraft.damage,Aircraft.Category,Make,Model,Number.of.Engines,Engine.Type,Purpose.of.flight,Weather.Condition,Broad.phase.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,United States,1948-10-24,Fatal(2),Destroyed,NaN,Stinson,108-3,1.0,Reciprocating,Personal,UNK,Cruise,2.0,0.0,0.0,0.0
1,United States,1962-07-19,Fatal(4),Destroyed,NaN,Piper,PA24-180,1.0,Reciprocating,Personal,UNK,Unknown,4.0,0.0,0.0,0.0
2,United States,1974-08-30,Fatal(3),Destroyed,NaN,Cessna,172M,1.0,Reciprocating,Personal,IMC,Cruise,3.0,NaN,NaN,NaN
3,United States,1977-06-19,Fatal(2),Destroyed,NaN,Rockwell,112,1.0,Reciprocating,Personal,IMC,Cruise,2.0,0.0,0.0,0.0
4,United States,1979-08-02,Fatal(1),Destroyed,NaN,Cessna,501,NaN,NaN,Personal,VMC,Approach,1.0,2.0,NaN,0.0


In [41]:
# check shape of dataset
subset_aviation.shape

(90348, 16)

In [42]:
# check dataset info
subset_aviation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 88663 non-null  object 
 1   Event.Date              88889 non-null  object 
 2   Injury.Severity         87889 non-null  object 
 3   Aircraft.damage         85695 non-null  object 
 4   Aircraft.Category       32287 non-null  object 
 5   Make                    88826 non-null  object 
 6   Model                   88797 non-null  object 
 7   Number.of.Engines       82805 non-null  float64
 8   Engine.Type             81812 non-null  object 
 9   Purpose.of.flight       82697 non-null  object 
 10  Weather.Condition       84397 non-null  object 
 11  Broad.phase.of.flight   61724 non-null  object 
 12  Total.Fatal.Injuries    77488 non-null  float64
 13  Total.Serious.Injuries  76379 non-null  float64
 14  Total.Minor.Injuries    76956 non-null

In [47]:
# check values in country column
subset_aviation['Country'].value_counts()

United States     82248
Brazil              374
Canada              359
Mexico              358
United Kingdom      344
                  ...  
Cambodia              1
Nauru                 1
Libya                 1
French Guiana         1
Seychelles            1
Name: Country, Length: 219, dtype: int64

In [48]:
# group data using groupby
make = subset_aviation.groupby(['Country'])
make.get_group('United States')

,Country,Event.Date,Injury.Severity,Aircraft.damage,Aircraft.Category,Make,Model,Number.of.Engines,Engine.Type,Purpose.of.flight,Weather.Condition,Broad.phase.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,United States,1948-10-24,Fatal(2),Destroyed,NaN,Stinson,108-3,1.0,Reciprocating,Personal,UNK,Cruise,2.0,0.0,0.0,0.0
1,United States,1962-07-19,Fatal(4),Destroyed,NaN,Piper,PA24-180,1.0,Reciprocating,Personal,UNK,Unknown,4.0,0.0,0.0,0.0
2,United States,1974-08-30,Fatal(3),Destroyed,NaN,Cessna,172M,1.0,Reciprocating,Personal,IMC,Cruise,3.0,NaN,NaN,NaN
3,United States,1977-06-19,Fatal(2),Destroyed,NaN,Rockwell,112,1.0,Reciprocating,Personal,IMC,Cruise,2.0,0.0,0.0,0.0
4,United States,1979-08-02,Fatal(1),Destroyed,NaN,Cessna,501,NaN,NaN,Personal,VMC,Approach,1.0,2.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90343,United States,2022-12-26,Minor,NaN,NaN,PIPER,PA-28-151,NaN,NaN,Personal,NaN,NaN,0.0,1.0,0.0,0.0
90344,United States,2022-12-26,NaN,NaN,NaN,BELLANCA,7ECA,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
90345,United States,2022-12-26,Non-Fatal,Substantial,Airplane,AMERICAN CHAMPION AIRCRAFT,8GCBC,1.0,NaN,Personal,VMC,NaN,0.0,0.0,0.0,1.0
90346,United States,2022-12-26,NaN,NaN,NaN,CESSNA,210N,NaN,NaN,Personal,NaN,NaN,0.0,0.0,0.0,0.0
